# 分类调用样例

In [ ]:
# 加载所需要的包
import os
import cv2
import glob
import base64
import time
import requests
import json
import urllib
import numpy as np
import matplotlib.pyplot as plt

<h2>定义调用规则</h2>
请根据您从我方获取的信息修改以下代码块

In [ ]:
# 朝厚服务请求地址，随api文档发送
base_url = "<服务请求地址>"

# 朝厚文件服务地址，随api文档发送
file_server_url = "<服务文件服务器地址>"

# 必须传入鉴权 Header。请保护好TOKEN!!! 如果泄露请立即联系我们重置，所有使用该TOKEN的任务都会向您的账户计费
zh_token = "<贵司服务Token, 随合同发送>" # 调用所有的API都必须传入token用作鉴权

user_group = "APIClient" # 用户组，一般为 APIClient

# 贵司user_id, 随api文档发送
user_id = "<贵司user_id>"

# 如果您收到了creds.json, 下面将直接读取
if os.path.exists('../../creds.json'):
    creds = json.load(open('../../creds.json', 'r'))
    base_url = creds['base_url']
    file_server_url = creds['file_server_url']
    zh_token = creds['zh_token']
    user_id = creds['user_id']
    print("loaded creds from creds.json")

## 图片分类

图片分类接口以及分类表请参考 https://www.chohotech.com/docs/cloud-zh/#/module/image-classification-2

In [ ]:
json_call = {
    "spec_version": "2.0-snapshot",
    "spec_name": "image-classification",
    "spec_group": "preprocessing",
    "user_group": user_group,
    "user_id": user_id
}

### 定义回调地址

回调会在工作流运行结束后向指定地址发送一个POST请求，请求内容为一个JSON，内含4个字段

1. workflow_id (str): 对应的workflow_id
2. metadata (dict): 您启动工作流时传入的metadata
3. success (bool): 工作流是否成功（true为成功）
4. reason (str or null): 如果success为true, 该项为null. 否则为失败原因的字符串表示

如果需要回调，请uncomment以下代码块

In [ ]:
# json_call['metadata'] = { #（可选）这里可以加入metadata, 回调时会附带该信息，字典每项限制128个字符
#     "case_id": "CH-123",
#     "case_name": "ABCDE"
# }
# json_call['notification'] =[ #（可选）这里可以加入回调地址
#     {"url": "https://www.baidu.com"} # 这里可以加入多个回调地址，每个都是{"url": "xxx"}格式
# ]

### 定义输入/输出块

输入块由JSON文件中的`input_data`定义。对于二维图片，我们支持以下**输入**文件格式

```
"jpg"
"jpeg"
"png"
```

对于二维图片，输入可以采用两种形式传入：

1. 先将文件传入我们的文件服务系统，再将文件指针写入JSON调用串并调用API
2. 直接在API的JSON调用串中传入base64编码后的二进制数据

我们**强烈推荐**使用第一种方式，理由如下：

1. base64编码将增大数据流大小，提高网络延迟
2. 为保障接口性能，朝厚将拒绝过大的API请求。因此对于大文件，第二种方式会调用失败（HTTP CODE 413)

In [ ]:
def upload_file(file_name):
    ext = file_name.split('.')[-1]
    data = open('../../data/' + file_name, 'rb').read()
    resp = requests.get(file_server_url + f"/scratch/{user_group}/{user_id}/upload_url?" +
                        f"postfix={ext}", # 必须指定 postfix, 即文件后缀名
                        headers={"X-ZH-TOKEN": zh_token}) # 获取带签名的上传地址
    resp.raise_for_status()

    upload_url = resp.text[1:-1] # 返回为一个单字符串JSON "string", 这里也可以用json.loads(resp.text)

    resp = requests.put(upload_url, data) # 上传至云储存服务不需要带鉴权头

    resp.raise_for_status()
    path = "/".join(urllib.parse.urlparse(upload_url).path.lstrip("/").split("/")[3:])
    urn = f"urn:zhfile:o:s:{user_group}:{user_id}:{path}"
    return urn

json_call["input_data"] = {
    "images":{
        fname: upload_file(fname) for fname in os.listdir('../../data/') if os.path.splitext(fname)[1] == '.jpg'
    }
}

### 提交请求

使用 `/run` POST方法提交请求

In [ ]:
headers = {
  "Content-Type": "application/json",
  "X-ZH-TOKEN": zh_token
}

url = base_url + '/run'

response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
response.raise_for_status()
create_result = response.json()
run_id = create_result['run_id']

# 打印任务号
print(run_id)

### 等待请求完成

使用轮询API等待请求完成，API地址为 `/run/{run_id}`

**强烈推荐使用回调方法**, 参考[定义回调地址](#定义回调地址)，回调方法无论工作流成功与否，在结束时均会向给定地址发送回调信息，您无需轮询结果

In [ ]:
# 第二步： 轮询状态
url = base_url + f"/run/{run_id}"

start_time = time.time()
while time.time()-start_time < 10: # 最多等3分钟
    time.sleep(0.3) # 轮询间隔
    response = requests.request("GET", url, headers=headers)
    result = response.json()
    if result['completed'] or result['failed']:
        break
    
if not result['completed']:
    if result['failed']:
        raise ValueError("API运行错误，原因： " + str(result['reason_public']))
    raise TimeoutError("API超时")

print("API运行时间： {}s".format(time.time()-start_time))

### 获取请求结果

使用 `/data/{run_id}` 获得该工作流所有输出数据，使用 `/data/{run_id}/{key}` 获取某一项数据

In [ ]:
url = base_url + f"/data/{run_id}"
response = requests.request("GET", url, headers=headers)
result = response.json()

In [ ]:
cls_name = {
    -1: "其他",
    0: "正面像",
    2: "正面微笑像",
    3: "正面咬合像",
    4: "覆盖像",
    5: "左侧咬合像",
    6: "下颌合面像",
    7: "右侧咬合像",
    8: "上颌合面像",
    9: "全景片",
    10: "侧位片",
    13: "右45度像",
    14: "右侧面微笑像",
    15: "左45度微笑像",
    16: "左45度像",
    17: "左侧面微笑像",
    18: "左侧面像",
    19: "右侧面像",
    20: "右45度微笑像",
    21: "小牙片"
}

for k, v in result['result']['results'].items():
    print(k, cls_name[v['predict_cls']])

## 上下颌分类

上下颌分类接口以及规则请参考 https://www.chohotech.com/docs/cloud-zh/#/module/jaw-classification-1

In [ ]:
json_call = {
    "spec_version": "1.0-snapshot",
    "spec_name": "jaw-classification",
    "spec_group": "preprocessing",
    "user_group": user_group,
    "user_id": user_id,
    "input_data": {
        "meshes": [
            {"type":"drc", "data": upload_file('upper_jaw_scan.drc')},
            {"type":"ply", "data": upload_file('lower_jaw_scan.ply')}
        ]
    }
}

url = base_url + '/run'

response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
response.raise_for_status()
create_result = response.json()
run_id = create_result['run_id']

# 打印任务号
print(run_id)

In [ ]:
# 轮询状态
url = base_url + f"/run/{run_id}"

start_time = time.time()
while time.time()-start_time < 10: # 最多等3分钟
    time.sleep(0.3) # 轮询间隔
    response = requests.request("GET", url, headers=headers)
    result = response.json()
    if result['completed'] or result['failed']:
        break
    
if not result['completed']:
    if result['failed']:
        raise ValueError("API运行错误，原因： " + str(result['reason_public']))
    raise TimeoutError("API超时")

print("API运行时间： {}s".format(time.time()-start_time))
url = base_url + f"/data/{run_id}"
response = requests.request("GET", url, headers=headers)
result = response.json()

In [ ]:
for i in result['result']['results']:
    print(
        "上颌" if i['upper_score'] > i['lower_score'] and i['upper_score'] > 0 else
            "下颌" if i['lower_score'] > i['upper_score'] and i['lower_score'] > 0 else "非半颌Mesh"
    )